In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import pickle
import numpy as np
from scipy import stats

In [3]:
dfs_real = {}
dfs_melted_real = {}
all_dfs_real = pd.DataFrame()
for n_groups in [2,3,4,5]:
    temp_dict_real = pickle.load(open(f'../50/experiment_results/final_scores_{n_groups}.pkl', 'rb'))
    temp_reform_real = {(outerKey, innerKey): values for outerKey, innerDict in temp_dict_real.items() for innerKey, values in innerDict.items()}
    dfs_real[n_groups] = pd.DataFrame(temp_reform_real)
    dfs_melted_real[n_groups] = dfs_real[n_groups].reset_index().melt(id_vars=['index'])
    dfs_melted_real[n_groups]['n_groups'] = n_groups
    all_dfs_real = all_dfs_real.append(dfs_melted_real[n_groups])

In [4]:
all_dfs_just_real = all_dfs_real.loc[all_dfs_real['index'] == 'real']

In [5]:
dfs = {}
dfs_melted = {}
all_dfs = pd.DataFrame()
for n_groups in [2,3,4,5]:
    dfs[n_groups] = {}
    dfs_melted[n_groups] = {}
    for fraction in [0.1, 0.3, 0.5, 0.7, 0.9, 0.998]:
        temp_dict = pickle.load(open(f'experiment_results/final_scores_{fraction}_{n_groups}.pkl', 'rb'))
        temp_reform = {(outerKey, innerKey): values for outerKey, innerDict in temp_dict.items() for innerKey, values in innerDict.items()}
        dfs[n_groups][fraction] = pd.DataFrame(temp_reform)
        dfs_melted[n_groups][fraction] = dfs[n_groups][fraction].reset_index().melt(id_vars=['index'])
        dfs_melted[n_groups][fraction]['fraction'] = fraction
        dfs_melted[n_groups][fraction]['n_groups'] = n_groups
        all_dfs = all_dfs.append(dfs_melted[n_groups][fraction])

In [6]:
all_dfs.replace('LSTM_MLP_with_CD', 'COSCI-GAN', inplace=True)
all_dfs.replace('LSTM_without_CD', 'COSCI-GAN without CD', inplace=True)
all_dfs.replace('real', 'Real', inplace=True)
all_dfs.replace('baseline', 'Baseline', inplace=True)

all_dfs_without_CD = all_dfs[(all_dfs['index'] != 'Real') & (all_dfs['fraction'] == 0.5)]

In [7]:
all_dfs_without_CD

,index,variable_0,variable_1,value,fraction,n_groups
0,Baseline,0,1,0.773171,0.5,2
1,GroupGAN,0,1,0.800000,0.5,2
2,GroupGAN without CD,0,1,0.802439,0.5,2
4,Baseline,0,2,0.790244,0.5,2
5,GroupGAN,0,2,0.770732,0.5,2
...,...,...,...,...,...,...
113,GroupGAN,4,8,0.868293,0.5,5
114,GroupGAN without CD,4,8,0.785366,0.5,5
116,Baseline,4,10,0.931707,0.5,5
117,GroupGAN,4,10,0.831707,0.5,5


In [8]:
all_dfs_total = all_dfs_without_CD.append(all_dfs_just_real)

In [9]:
all_dfs_total_5 = all_dfs_total.loc[all_dfs_total['n_groups'] == 5]

In [10]:
all_dfs_total_5.rename(columns={'index': 'model', 'variable_1': 'ratio'}, inplace=True)
all_dfs_total_5.rename(columns={'value': 'score'}, inplace=True)
fig = px.box(all_dfs_total_5, x='ratio', y='score', color='model')
fig

/Users/aliseyfi/opt/miniconda3/envs/research/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [11]:
from copy import deepcopy
new_all_dfs = deepcopy(all_dfs)
new_all_dfs['fraction+n_groups'] = new_all_dfs['fraction'] + new_all_dfs['n_groups']

In [12]:
fig = px.box(new_all_dfs, x='fraction+n_groups', y='value', color='index')
fig.update_xaxes(tickvals=[1.1, 1.3, 1.5, 1.7, 1.9, 2, 2.1, 2.3, 2.5, 2.7, 2.9, 3, 3.1, 3.3, 3.5, 3.7, 3.9, 4, 4.1, 4.3, 4.5, 4.7, 4.9, 5, 5.1, 5.3, 5.5, 5.7, 5.9, 6])
fig.show()

In [23]:
def add_anootation(fig, x0, x1, symbol, y_offset=0.0):
    fig.add_shape(type="line",
        xref="x", yref="y"+" domain",
        x0=x0, y0=1.00 + y_offset, 
        x1=x0, y1=0.98 + y_offset,
        line=dict(color='black', width=2,)
    )
    # Horizontal line
    fig.add_shape(type="line",
        xref="x", yref="y"+" domain",
        x0=x0, y0=1.00 + y_offset,
        x1=x1, y1=1.00 + y_offset,
        line=dict(color='black', width=2,)
    )
    # Vertical line
    fig.add_shape(type="line",
        xref="x", yref="y"+" domain",
        x0=x1, y0=1.00 + y_offset,
        x1=x1, y1=0.98 + y_offset,
        line=dict(color='black', width=2,)
    )
    ## add text at the correct x, y coordinates
    ## for bars, there is a direct mapping from the bar number to 0, 1, 2...
    if symbol != 'ns':
        fig.add_annotation(dict(font=dict(color='black',size=14),
            x=(x0 + x1)/2,
            y=1.03 + y_offset,
            showarrow=False,
            text=symbol,
            textangle=0,
            xref="x",
            yref="y"+" domain"
        ))
    else:
        fig.add_annotation(dict(font=dict(color='black',size=14),
            x=(x0 + x1)/2,
            y=1.04 + y_offset,
            showarrow=False,
            text=symbol,
            textangle=0,
            xref="x",
            yref="y"+" domain"
        ))
    return fig

def symbol_generator(data_1, data_2):
    pvalue = stats.ttest_ind(
        data_1,
        data_2,
        equal_var=False,
    )[1]
    if pvalue >= 0.05:
        symbol = 'ns'
    elif pvalue >= 0.01: 
        symbol = '*'
    elif pvalue >= 0.001:
        symbol = '**'
    else:
        symbol = '***'
    return symbol

In [24]:
all_dfs_total

,index,variable_0,variable_1,value,fraction,n_groups
0,Baseline,0,1,0.773171,0.5,2
1,GroupGAN,0,1,0.800000,0.5,2
2,GroupGAN without CD,0,1,0.802439,0.5,2
4,Baseline,0,2,0.790244,0.5,2
5,GroupGAN,0,2,0.770732,0.5,2
...,...,...,...,...,...,...
103,real,4,2,NaN,NaN,5
107,real,4,4,NaN,NaN,5
111,real,4,6,NaN,NaN,5
115,real,4,8,NaN,NaN,5


In [25]:
fig = px.box(all_dfs_total, x='n_groups', y='value', color='index', labels={'index': 'Model', 'value': 'Accuracy', 'n_groups': 'Number of channels'}, color_discrete_sequence=['red', 'green', 'yellow', 'blue'])

# Group 2
data_1 = all_dfs_total[(all_dfs_total['n_groups']==2) & (all_dfs_total['index']=='COSCI-GAN')]['value']
data_2 = all_dfs_total[(all_dfs_total['n_groups']==2) & (all_dfs_total['index']=='Baseline')]['value']

fig = add_anootation(fig, 1.74, 1.90, symbol_generator(data_1, data_2))

data_2 = all_dfs_total[(all_dfs_total['n_groups']==2) & (all_dfs_total['index']=='COSCI-GAN without CD')]['value']
fig = add_anootation(fig, 1.93, 2.08, symbol_generator(data_1, data_2))

data_2 = all_dfs_total[(all_dfs_total['n_groups']==2) & (all_dfs_total['index']=='Real')]['value']
fig = add_anootation(fig, 1.93, 2.27, symbol_generator(data_1, data_2), y_offset=0.045)

# Group 3
data_1 = all_dfs_total[(all_dfs_total['n_groups']==3) & (all_dfs_total['index']=='COSCI-GAN')]['value']
data_2 = all_dfs_total[(all_dfs_total['n_groups']==3) & (all_dfs_total['index']=='Baseline')]['value']

fig = add_anootation(fig, 2.74, 2.90, symbol_generator(data_1, data_2))

data_2 = all_dfs_total[(all_dfs_total['n_groups']==3) & (all_dfs_total['index']=='COSCI-GAN without CD')]['value']
fig = add_anootation(fig, 2.93, 3.08, symbol_generator(data_1, data_2))

data_2 = all_dfs_total[(all_dfs_total['n_groups']==3) & (all_dfs_total['index']=='Real')]['value']
fig = add_anootation(fig, 2.93, 3.27, symbol_generator(data_1, data_2), y_offset=0.045)

# Group 4
data_1 = all_dfs_total[(all_dfs_total['n_groups']==4) & (all_dfs_total['index']=='COSCI-GAN')]['value']
data_2 = all_dfs_total[(all_dfs_total['n_groups']==4) & (all_dfs_total['index']=='Baseline')]['value']

fig = add_anootation(fig, 3.74, 3.90, symbol_generator(data_1, data_2))

data_2 = all_dfs_total[(all_dfs_total['n_groups']==4) & (all_dfs_total['index']=='COSCI-GAN without CD')]['value']
fig = add_anootation(fig, 3.93, 4.08, symbol_generator(data_1, data_2))

data_2 = all_dfs_total[(all_dfs_total['n_groups']==4) & (all_dfs_total['index']=='Real')]['value']
fig = add_anootation(fig, 3.93, 4.27, symbol_generator(data_1, data_2), y_offset=0.045)

# Group 5
data_1 = all_dfs_total[(all_dfs_total['n_groups']==5) & (all_dfs_total['index']=='COSCI-GAN')]['value']
data_2 = all_dfs_total[(all_dfs_total['n_groups']==5) & (all_dfs_total['index']=='Baseline')]['value']

fig = add_anootation(fig, 4.74, 4.90, symbol_generator(data_1, data_2))

data_2 = all_dfs_total[(all_dfs_total['n_groups']==5) & (all_dfs_total['index']=='COSCI-GAN without CD')]['value']
fig = add_anootation(fig, 4.93, 5.08, symbol_generator(data_1, data_2))

data_2 = all_dfs_total[(all_dfs_total['n_groups']==5) & (all_dfs_total['index']=='Real')]['value']
fig = add_anootation(fig, 4.93, 5.27, symbol_generator(data_1, data_2), y_offset=0.045)


fig.update_layout(yaxis_range=[0.45,1])
fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1],
    ))
fig.update_layout(yaxis_title=None)
fig.update_layout(margin=dict(l=0, r=15, b=0, t=38), font=dict(size=23), width=700, height=600, showlegend=False)
fig.show()


In [42]:
fig.write_image("augmentation_with_without.pdf")

In [43]:
all_dfs = []
for n_groups in [2,3,4,5]:
    for fraction in [0.3]:#0.1, 0.3, 0.5, 0.7, 0.9, 0.998]:
        temp_df = dfs[n_groups][fraction].T.droplevel(0)
        temp_df['amount'] = dfs[n_groups][fraction].T.droplevel(0).index
        temp_df['n_groups'] = n_groups
        temp_df['fraction'] = fraction
        all_dfs.append(temp_df)

all_dfs = pd.concat(all_dfs)

all_dfs["n_groups"] = all_dfs["n_groups"].astype(str)

In [45]:
fig = px.scatter(all_dfs, x='LSTM_without_CD', y='LSTM_MLP_with_CD', color='n_groups', color_discrete_sequence=["#cc0099", "orange", "green", "blue"], labels={'n_groups': '# channels', 'baseline': 'Baseline', 'LSTM_MLP_with_CD': 'COSCI-GAN'})
fig.update_layout(width = 550,
                  height = 400,margin=dict(l=0, r=0, b=0, t=10), font=dict(size=15))
fig.update_xaxes(range=[0.45, 1.0])
fig.update_yaxes(range=[0.45, 1.0])
fig.update_xaxes(tickvals=[0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0])
fig.update_yaxes(tickvals=[0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0])
fig.update_traces(marker={'size': 10})

fig.add_trace(go.Scatter(
    x = [0.45,1.1],
    y = [0.45,1.1],
    showlegend=False,
    line=dict(color='red')
))


fig.add_trace(go.Scatter(
    x = [0.45,1.1],
    y = [0.55,1.2],
    showlegend=False,
    line=dict(dash='dash', color='black')
))


fig.add_trace(go.Scatter(
    x = [0.45,1.1],
    y = [0.65,1.3],
    showlegend=False,
    line=dict(dash='dash', color='black')
))

fig.add_trace(go.Scatter(
    x = [0.45,1.1],
    y = [0.75,1.4],
    showlegend=False,
    line=dict(dash='dash', color='black')
))

fig.add_trace(go.Scatter(
    x = [0.45,1.1],
    y = [0.35,1.0],
    showlegend=False,
    line=dict(dash='dash', color='black')
))

In [123]:
fig.write_image("scatter.pdf")
